In [16]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import *
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


# Goal: Simultaneously Estimate Translation, Rotation, and Distortion

## $\mathbf{A} = [\hat{X}_{ij}, \hat{m}_{ij}] = 
\begin{bmatrix}
% x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
\end{bmatrix}$

We can use newton-raphson to find A

<!-- ## $y_i = \mathbf{h}(y_j, \hat{X}_{ij}, \hat{m}_{ij}) + \mathbf{H}_m \delta m + \mathbf{H}_x \delta x + \text{H.O.T.}$ -->

## $y_i = \mathbf{h}(y_j, \hat{A}_{ij}) + \mathbf{H}_A \delta A + \text{H.O.T.}$

## $\mathbf{H}_A \in \mathbb{R}^{4N \times 12} $



# I'm starting to think that this construction of H is double counting 

### $\mathbf{H}_A = [H_X, H_m]$

# Run Demo

In [17]:
# load point cloud
# # no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# # m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #actual motion
# m_hat = np.array([-3., 0., 0., 0., 0., 0.1]) #test wrap around

# # movement in x
# old_cloud =  np.load("sample_data/paper_figures/case2/raw_frame_3.npy") 
# m_hat = np.array([3, 0, 0., 0., 0., 0])
# # m_hat = np.array([3, 0, 0., 0., 0., -0.2]) #FOR DEBUG-- deform just a little
# gt =  np.load("sample_data/paper_figures/case2/base_vel_2.npy")

# movement in x, y, & yaw
old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
# m_hat = np.array([3, -1, 0., 0., 0., -1])
m_hat = np.array([3, -1, 0., 0., 0., -0.86]) #FOR DEBUG-- deform a little extra
# m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #FOR DEBUG
gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# print(gt) 

# period_lidar = 1
# t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
# m_hat = m_hat*t_scale
# # m_hat[-1] = m_hat[-1]*t_scaled
# print(m_hat)

#downsample
old_cloud = old_cloud[::5,:]

In [18]:
from remove_motion_basic import linear_correction_old as lc 

#apply ground truth distortion according to m_hat
new_cloud = lc(old_cloud, m_hat) 
#set ground truth transform between clouds
# X_gt = np.array([0.5, 0.5, 0.03, 0.004, 0.05, 0.06])
X_gt = np.array([0.5, 0.5, 0.03, 0.0, 0.0, 0.0])

# #remove ground plane
# old_cloud = old_cloud[old_cloud[:,2] > -1] 
# new_cloud = new_cloud[new_cloud[:,2] > -1] 

# Rotate + Translate new point cloud
trans = X_gt[:3]
rot = R_tf(X_gt[3:]).numpy()
new_cloud = (new_cloud @ rot) + trans

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(old_cloud, c = "#CB2314"))
disp.append(Points(new_cloud, c = "#3F5151"))
plt.show(disp, "raw point clouds")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

### Attempt to solve with basic Newton-Raphson

In [19]:
from linear_corrector import LC
pc1 = old_cloud
pc2  = new_cloud
m_hat0 = np.array([0, 0, 0, 0, 0, 0.])
dc = LC(cloud1 = pc2, cloud2 = pc1, fid = 50, niter = 50, draw = True, m_hat0 = m_hat0,  mnp = 25, RM = False)
ViewInteractiveWidget(dc.plt.window)


~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
took 0.015839099884033203 sec  to apply motion profile
M 
 [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0

M 
 [[ 3.99441105e-04 -6.28432007e-03  1.60982007e-03  4.24143213e-05
  -2.45829022e-05 -5.56969016e-03]
 [ 4.06610733e-04 -6.39711826e-03  1.63871497e-03  4.31756222e-05
  -2.50241444e-05 -5.66966135e-03]
 [ 4.08419185e-04 -6.42557024e-03  1.64600336e-03  4.33676511e-05
  -2.51354424e-05 -5.69487788e-03]
 [ 6.82874085e-04 -1.07435095e-02  2.75210636e-03  7.25104163e-05
  -4.20262879e-05 -9.52179688e-03]
 [ 6.84931633e-04 -1.07758805e-02  2.76039865e-03  7.27288953e-05
  -4.21529160e-05 -9.55048673e-03]
 [ 6.89574656e-04 -1.08489282e-02  2.77911088e-03  7.32219109e-05
  -4.24386628e-05 -9.61522769e-03]
 [ 9.47028290e-04 -1.48993902e-02  3.81669570e-03  1.00559411e-04
  -5.82831951e-05 -1.32050860e-02]
 [ 9.49552859e-04 -1.49391087e-02  3.82687017e-03  1.00827480e-04
  -5.84385653e-05 -1.32402879e-02]
 [ 9.52129906e-04 -1.49796528e-02  3.83725613e-03  1.01101121e-04
  -5.85971651e-05 -1.32762214e-02]
 [ 1.22284233e-03 -1.92387126e-02  4.92827629e-03  1.29846495e-04
  -7.52576866e-05 -1.

M 
 [[ 1.71327052e-05 -3.82659652e-03  2.49079166e-03  1.78119651e-04
  -1.16326162e-04 -1.27131884e-02]
 [ 1.71321702e-05 -3.82647703e-03  2.49071388e-03  1.78114088e-04
  -1.16322530e-04 -1.27127914e-02]
 [ 1.71461914e-05 -3.82960867e-03  2.49275232e-03  1.78259860e-04
  -1.16417730e-04 -1.27231958e-02]
 [ 2.90321992e-05 -6.48435326e-03  4.22076718e-03  3.01832381e-04
  -1.97120320e-04 -2.15431139e-02]
 [ 2.90239631e-05 -6.48251372e-03  4.21956980e-03  3.01746754e-04
  -1.97064399e-04 -2.15370024e-02]
 [ 2.89375908e-05 -6.46322243e-03  4.20701279e-03  3.00848787e-04
  -1.96477955e-04 -2.14729105e-02]
 [ 4.06152599e-05 -9.07143448e-03  5.90473890e-03  4.22255321e-04
  -2.75765985e-04 -3.01382325e-02]
 [ 4.06473384e-05 -9.07859923e-03  5.90940255e-03  4.22588824e-04
  -2.75983789e-04 -3.01620362e-02]
 [ 4.06398655e-05 -9.07693014e-03  5.90831611e-03  4.22511131e-04
  -2.75933049e-04 -3.01564909e-02]
 [ 5.18597489e-05 -1.15828956e-02  7.53948830e-03  5.39158310e-04
  -3.52112846e-04 -3.

M 
 [[ 1.67102211e-02  7.30372462e-03  2.87535067e-03  2.53978150e-04
  -2.10691094e-04 -1.79981322e-02]
 [ 1.66485115e-02  7.27675251e-03  2.86473222e-03  2.53040228e-04
  -2.09913027e-04 -1.79316664e-02]
 [ 1.65513975e-02  7.23430580e-03  2.84802168e-03  2.51564195e-04
  -2.08688564e-04 -1.78270675e-02]
 [ 2.80847443e-02  1.22753157e-02  4.83258050e-03  4.26859189e-04
  -3.54106955e-04 -3.02493269e-02]
 [ 2.78739267e-02  1.21831712e-02  4.79630482e-03  4.23654978e-04
  -3.51448857e-04 -3.00222608e-02]
 [ 2.81311969e-02  1.22956192e-02  4.84057366e-03  4.27565220e-04
  -3.54692653e-04 -3.02993597e-02]
 [ 3.92673650e-02  1.71630297e-02  6.75678938e-03  5.96823505e-04
  -4.95103209e-04 -4.22938285e-02]
 [ 3.93187173e-02  1.71854748e-02  6.76562564e-03  5.97604007e-04
  -4.95750685e-04 -4.23491387e-02]
 [ 3.93281701e-02  1.71896064e-02  6.76725219e-03  5.97747680e-04
  -4.95869871e-04 -4.23593200e-02]
 [ 5.04395373e-02  2.20461769e-02  8.67920040e-03  7.66629018e-04
  -6.35967725e-04 -5.

M 
 [[ 3.42571667e-02  1.41822194e-02  3.02943777e-03  3.04627755e-04
  -2.93474285e-04 -2.11894515e-02]
 [ 3.36386556e-02  1.39261603e-02  2.97474145e-03  2.99127719e-04
  -2.88175624e-04 -2.08068772e-02]
 [ 3.46390579e-02  1.43403196e-02  3.06320926e-03  3.08023676e-04
  -2.96745870e-04 -2.14256667e-02]
 [ 5.74995693e-02  2.38044061e-02  5.08481534e-03  5.11308038e-04
  -4.92587291e-04 -3.55658231e-02]
 [ 5.72110437e-02  2.36849586e-02  5.05930038e-03  5.08742359e-04
  -4.90115551e-04 -3.53873583e-02]
 [ 5.75614969e-02  2.38300437e-02  5.09029174e-03  5.11858722e-04
  -4.93117813e-04 -3.56041279e-02]
 [ 8.06250191e-02  3.33781752e-02  7.12985053e-03  7.16948333e-04
  -6.90698388e-04 -4.98698548e-02]
 [ 8.05220020e-02  3.33355269e-02  7.12074049e-03  7.16032266e-04
  -6.89815861e-04 -4.98061345e-02]
 [ 8.07691262e-02  3.34378345e-02  7.14259424e-03  7.18229788e-04
  -6.91932924e-04 -4.99589909e-02]
 [ 8.44478661e-02  3.49608062e-02  7.46791342e-03  7.50942542e-04
  -7.23447951e-04 -5.

...

M 
 [[ 2.53645568e-01  9.23349094e-02  1.89684013e-03  3.31667957e-04
  -9.08692481e-04 -3.18905383e-02]
 [ 2.55089797e-01  9.28606539e-02  1.90764052e-03  3.33556437e-04
  -9.13866473e-04 -3.20721194e-02]
 [ 2.49918205e-01  9.09780327e-02  1.86896575e-03  3.26794042e-04
  -8.95339095e-04 -3.14219016e-02]
 [ 2.82210388e-01  1.02733396e-01  2.11045670e-03  3.69019429e-04
  -1.01102676e-03 -3.54819572e-02]
 [ 2.78421305e-01  1.01354051e-01  2.08212076e-03  3.64064809e-04
  -9.97452263e-04 -3.50055605e-02]
 [ 4.20728488e-01  1.53158311e-01  3.14633795e-03  5.50146249e-04
  -1.50727180e-03 -5.28976637e-02]
 [ 4.21511772e-01  1.53443451e-01  3.15219560e-03  5.51170475e-04
  -1.51007794e-03 -5.29961450e-02]
 [ 4.16448729e-01  1.51600345e-01  3.11433259e-03  5.44550019e-04
  -1.49193945e-03 -5.23595750e-02]
 [ 4.26778673e-01  1.55360767e-01  3.19158311e-03  5.58057495e-04
  -1.52894676e-03 -5.36583458e-02]
 [ 4.31343495e-01  1.57022505e-01  3.22572027e-03  5.64026475e-04
  -1.54530037e-03 -5.

M 
 [[ 2.70113196e-01  9.16976752e-02  1.76152194e-03  3.30929659e-04
  -9.58143902e-04 -3.20002782e-02]
 [ 2.69966460e-01  9.16478617e-02  1.76056502e-03  3.30749886e-04
  -9.57623403e-04 -3.19828945e-02]
 [ 2.65107220e-01  8.99982530e-02  1.72887586e-03  3.24796579e-04
  -9.40386734e-04 -3.14072208e-02]
 [ 2.95177170e-01  1.00206360e-01  1.92497467e-03  3.61636832e-04
  -1.04705068e-03 -3.49696042e-02]
 [ 3.06410401e-01  1.04019803e-01  1.99823131e-03  3.75399245e-04
  -1.08689713e-03 -3.63004038e-02]
 [ 2.93503088e-01  9.96380453e-02  1.91405728e-03  3.59585827e-04
  -1.04111239e-03 -3.47712760e-02]
 [ 4.47449125e-01  1.51899445e-01  2.91800424e-03  5.48193087e-04
  -1.58718884e-03 -5.30092447e-02]
 [ 4.47006689e-01  1.51749247e-01  2.91511892e-03  5.47651035e-04
  -1.58561943e-03 -5.29568293e-02]
 [ 4.40536138e-01  1.49552633e-01  2.87292174e-03  5.39723628e-04
  -1.56266714e-03 -5.21902639e-02]
 [ 4.54206065e-01  1.54193282e-01  2.96206909e-03  5.56471364e-04
  -1.61115702e-03 -5.

M 
 [[ 2.86625578e-01  9.28841242e-02  1.65099666e-03  3.20551020e-04
  -1.00399203e-03 -3.18877700e-02]
 [ 2.85458929e-01  9.25060589e-02  1.64427662e-03  3.19246284e-04
  -9.99905492e-04 -3.17579775e-02]
 [ 2.80526205e-01  9.09075562e-02  1.61586356e-03  3.13729715e-04
  -9.82627148e-04 -3.12092004e-02]
 [ 3.10590687e-01  1.00650277e-01  1.78903847e-03  3.47352676e-04
  -1.08793701e-03 -3.45539447e-02]
 [ 3.19024557e-01  1.03383364e-01  1.83761854e-03  3.56784791e-04
  -1.11747917e-03 -3.54922326e-02]
 [ 3.08582565e-01  9.99995238e-02  1.77747146e-03  3.45106870e-04
  -1.08090296e-03 -3.43305365e-02]
 [ 4.72153140e-01  1.53006341e-01  2.71965698e-03  5.28037907e-04
  -1.65385795e-03 -5.25281478e-02]
 [ 4.73237074e-01  1.53357601e-01  2.72590057e-03  5.29250138e-04
  -1.65765475e-03 -5.26487380e-02]
 [ 4.67182360e-01  1.51395506e-01  2.69102471e-03  5.22478778e-04
  -1.63644630e-03 -5.19751368e-02]
 [ 4.79118060e-01  1.55263399e-01  2.75977573e-03  5.35827206e-04
  -1.67825467e-03 -5.

M 
 [[ 2.98430812e-01  9.46437219e-02  1.46532094e-03  3.08200364e-04
  -1.05441650e-03 -3.14802518e-02]
 [ 2.98106660e-01  9.45409212e-02  1.46372933e-03  3.07865600e-04
  -1.05327120e-03 -3.14460584e-02]
 [ 2.95211147e-01  9.36226444e-02  1.44951211e-03  3.04875299e-04
  -1.04304077e-03 -3.11406225e-02]
 [ 3.22568911e-01  1.02298828e-01  1.58384108e-03  3.33128657e-04
  -1.13970129e-03 -3.40264817e-02]
 [ 3.33501856e-01  1.05766079e-01  1.63752278e-03  3.44419508e-04
  -1.17832960e-03 -3.51797535e-02]
 [ 3.13872217e-01  9.95407768e-02  1.54113956e-03  3.24147265e-04
  -1.10897411e-03 -3.31091028e-02]
 [ 4.94504888e-01  1.56826243e-01  2.42806150e-03  5.10693200e-04
  -1.74718592e-03 -5.21633082e-02]
 [ 4.94924551e-01  1.56959334e-01  2.43012209e-03  5.11126601e-04
  -1.74866867e-03 -5.22075767e-02]
 [ 4.89737590e-01  1.55314353e-01  2.40465367e-03  5.05769838e-04
  -1.73034209e-03 -5.16604254e-02]
 [ 4.98748722e-01  1.58172124e-01  2.44889910e-03  5.15075962e-04
  -1.76218024e-03 -5.

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [20]:
# ##DEBUG: make sure that correspondeces between old and new cloud are good
# pc1_theta = dc.c2s(pc1)[:,1]
# pc2_theta = dc.c2s(pc2)[:,1]
# from matplotlib import pyplot as plt
# fig, ax = plt.subplots()
# ax.plot(pc1_theta)
# ax.plot(pc2_theta)

In [21]:
A_hat = np.array([0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
skip = 1 #50
y_i = new_cloud[::skip] #baseline
y_j = old_cloud[::skip] #distorted cloud

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i[:,:3], c = "#2c7c94 ", alpha = 1, r=3.5))

runlen = 3
for count in range(runlen):
    
    print("~~~~ iteration ", count, "~~~~~~~~~~~")
    print("A_hat = \n", np.round(A_hat[:6],4), "\n", np.round(A_hat[6:],4)) 
    
    #decompose A_hat into X_hat and m_hat
    X_hat = A_hat[:6] 
    m_hat = A_hat[6:] 
#     X_hat = np.array([0, 0, 0, 0, 0, 0.]) #for debug
#     m_hat = np.array([0, 0, 0, 0, 0, 0.]) #for debug
    m_hat = 0.1*m_hat
    
    #apply last estimate of distortion correction
#     print("m_hat", m_hat)
    y_j_undistort = lc(y_j, m_hat)

    #apply last rigid transform
    rot = R_tf(-X_hat[3:]).numpy()
    trans = X_hat[:3]
    y_j_undistort = (y_j_undistort @ rot) + trans
    
#     print("rot: \n", rot,"\n trans: \n", trans)  
#     print("\n y_i \n",np.shape(y_i), "\n", y_i[:3])
#     print("y_j_undistort \n",np.shape(y_j_undistort), "\n", y_j_undistort[:3])

    
    #get H = [H_X, H_m]
    #get jacobain of distortion correction function
    H_m = dc.get_H_m(y_j_undistort, m_hat) 
    print("\n H_m:", np.shape(H_m), "\n", H_m[:10])
    
    #get jacobian of rigid transform function 
    H_x = jacobian_tf(tf.transpose(tf.convert_to_tensor(y_j_undistort, tf.float32)), tf.convert_to_tensor(X_hat[3:], tf.float32)) # shape = [num of corr * 3, 6]
    #need to append on a row of zeros since we are working with homogeneous coordinates!
    H_x = tf.reshape(H_x, (tf.shape(H_x)[0]//3, 3, 6)) # -> need shape [#corr//4, 4, 6]
#     print("\n H_x:", np.shape(H_x), "\n", H_x[0])
    H_x = tf.concat([H_x, tf.zeros([len(H_x),1,6])], axis = 1)
    H_x = tf.reshape(H_x, (-1, 6))
    print("\n H_x:", np.shape(H_x), "\n", H_x[:10])
    H_x = H_x.numpy()
        
    #delta_A =  ((H^T*H)^-1)(H^T)(yi - yj_undistort)
    residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
                np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1)).flatten()
#     print("residual", np.shape(residual), "\n", residual)
    
    H = np.append(H_x, H_m, axis = 1)
#     print("H: \n", np.shape(H))
    
    delta_A = np.linalg.pinv(H.T @ H) @ H.T @ residual
    print("\n delta_A \n", np.round(delta_A[:6], 5), "\n", np.round(delta_A[6:], 5))
    #augment rigid transform components
    A_hat[:6] -=   delta_A[:6]
    #augment distortion components
    A_hat[6:9] -= delta_A[6:9]
    A_hat[9:] += delta_A[9:]

    #plot updated cloud2
    color = [0.5 + count/(runlen*2), 1 - count/runlen, count/runlen]
    disp.append(Points(y_j_undistort[:,:3], c = color, r=3.5))
#     disp.append(Points(y_j_undistort[:,:3], c = "#a65852 ", alpha = (count+1)/(runlen+1), r=3.5))

    
plt.show(disp, "12 State Solution")
ViewInteractiveWidget(plt.window)

~~~~ iteration  0 ~~~~~~~~~~~
A_hat = 
 [0. 0. 0. 0. 0. 0.] 
 [0. 0. 0. 0. 0. 0.]
M 
 [[-0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0.]
 ...
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]

 H_m: (66256, 6) 
 [[-1.          0.          0.          0.         -1.47598863  0.01319261]
 [ 0.         -1.          0.          1.47598863  0.          2.48878336]
 [ 0.          0.         -1.         -0.01319261 -2.48878336  0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [-1.          0.          0.          0.         -1.45496595  0.01488573]
 [ 0.         -1.          0.          1.45496595  0.          2.8081913 ]
 [ 0.          0.         -1.         -0.01488573 -2.8081913   0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [-1.          0.          0.          0.         -1.47213745  0.01743821]
 [ 0.         -1.          0.          1.47213745  

ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)